# Implementing WaveNet LSTM
I will start by checking weights etc. over here

In [2]:
%pip install torchsummary

Note: you may need to restart the kernel to use updated packages.


In [2]:
import torch
from torch import nn
from torch.nn import functional as F
import numpy as np
from models.wavenet.wavenet import WaveLayer, WaveBlock, WaveNet
from torchsummary import summary

In [21]:
# Making random training data
train_x = torch.rand((5, 22, 15000))

In [22]:
wavenet = WaveNet()

In [23]:
wavenet.forward(train_x)

tensor([[0.2326, 0.3885, 0.4845, 0.5420, 0.5768, 0.5984, 0.6123, 0.6217, 0.6281,
         0.6327, 0.0000, 0.0000, 0.0000, 0.6416, 0.6426, 0.0000, 0.0000, 0.0000,
         0.6448, 0.0000, 0.6453, 0.6454, 0.0000, 0.0000, 0.6457, 0.0000, 0.0000,
         0.6458, 0.6459, 0.0000, 0.6459, 0.0000, 0.6459, 0.6459, 0.0000, 0.6460,
         0.0000, 0.6460, 0.0000, 0.6460, 0.0000, 0.0000, 0.6460, 0.0000, 0.6460,
         0.6460, 0.6460, 0.6460, 0.0000, 0.0000, 0.6460, 0.0000, 0.6460, 0.0000,
         0.6460, 0.6460, 0.0000, 0.0000, 0.0000, 0.6460, 0.0000, 0.0000, 0.0000,
         0.0000],
        [0.0000, 0.3885, 0.4845, 0.0000, 0.5768, 0.5984, 0.0000, 0.6217, 0.6281,
         0.0000, 0.0000, 0.6384, 0.0000, 0.0000, 0.0000, 0.6434, 0.6440, 0.6444,
         0.6448, 0.0000, 0.0000, 0.0000, 0.0000, 0.6456, 0.6457, 0.6458, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.6459, 0.0000, 0.6459, 0.0000,
         0.6460, 0.0000, 0.6460, 0.6460, 0.6460, 0.6460, 0.0000, 0.6460, 0.6460,
         0

In [24]:
# Now I will check different things here
block1 = WaveBlock(22, 16, 3, 8)
block2 = WaveBlock(16, 32, 3, 5)
block3 = WaveBlock(32, 64, 3, 3)
block4 = WaveBlock(64, 64, 2, 2)
train_x = block1.forward(train_x)
train_x = F.avg_pool1d(train_x, 10)

print("train_x size now: ", train_x.shape)
train_x = block2.forward(train_x)
train_x = F.avg_pool1d(train_x, 10)
print("train_x size now: ", train_x.shape)
train_x = block3.forward(train_x)
train_x = F.avg_pool1d(train_x, 10)
print("train_x size now: ", train_x.shape)
train_x = block4.forward(train_x)
train_x = F.avg_pool1d(train_x, 2)
print("train_x size now: ", train_x.shape)

train_x size now:  torch.Size([5, 16, 1500])
train_x size now:  torch.Size([5, 32, 150])
train_x size now:  torch.Size([5, 64, 15])
train_x size now:  torch.Size([5, 64, 7])


In [25]:
train_x.shape

torch.Size([5, 64, 7])

In [26]:
# Operates on batch - seq - features
# We have tensor of 1, 64, 5
# we will resize it to 1, 5, 64
lstm = nn.LSTM(7, 64, batch_first=True)

In [27]:
train_x.shape

torch.Size([5, 64, 7])

In [28]:
y = lstm(train_x)

In [29]:
y[0].shape

torch.Size([5, 64, 64])

In [32]:
y[1][1][0].shape

torch.Size([5, 64])

In [107]:
y[0].squeeze(-1).shape

torch.Size([1, 64])